# 4. Wstępne przetwarzanie danych


Przed przystąpieniem do analizy dane należy odpowiednio przygotować
* usunięcie błędów wynikające z metody pomiaru lub niepoprawnej akwizycji
* usunięcie wartości niezgodnych ze zbiorem danych i naszą wiedzą o domenie problemu
* wartości odstające (_outlayers_)
* wartości brakujące
* powtórzenia pomiarów (duplikaty)
* przygotowanie zmiennych: standaryzacja, normalizacja, uciąglanie, dyskretyzacja, ...

## Dane Iris

Dane Iris zawierają pomiary rozmiarów płatków (petal) oraz rozmiarów liści kielicha (sepal) dokonanych dla 3 odmian Irysów: Setosa, Virginica i Versicolor

<img src="https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Machine+Learning+R/iris-machinelearning.png" alt="drawing" width="400"/>

Dane można porać z adresu https://www.is.umk.pl/~grochu/wdm/files/iris-data.csv lub z katalogu z danymi w repozytorium ``dane/iris-data.csv``


In [ ]:
import pandas as pd

iris = pd.read_csv('https://www.is.umk.pl/~grochu/wdm/files/iris-data.csv')

print('Ilość przypadków = %d' % (iris.shape[0]))
print('Ilość zmiennych = %d' % (iris.shape[1]))
iris.head()        # wyswielimy pięc pierwszch wierszy

## Brakujące wartości

Na pierwszy rzut oka wszystko wydaje się być w porządku. Niemniej jednak pierwsze co powinniśmy sprawdzić to to czy w pliku nie ma braków danych. W tabeli Pandas takie wartości są reprezentowane za pomocą wartości `null`. Sprawdźmy, czy mamy takie dane w pliku za pomocą metody `isnull()`

In [ ]:
iris.isnull()

In [ ]:
iris.isnull().values.any()

Wygląda na to, że w danych są brakujące wartości. Sprawdźmy ile ich jest w każdej ze zmiennych.

In [ ]:
iris.isnull().sum()

Zobaczmy także jakiego typu są zmienne.

In [ ]:
iris.info()

Pierwsza zmienna powinna zawierać wartości numeryczne a jest reprezentowana jako `object`. Najprawdopodobniej w pliku występuje pewna wartość, która nie jest liczbą, dlatego Pandas zaimportował te wartości do typu `object` (w postaci napisów). Spróbujmy zamienić tą zmienną na zmienną numeryczną za pomocą metody `to_numeric()`. Argument  `errors='coerce'` sprawia, że  wszystkie problematyczne wartości zostaną zamienione na NaN.

In [ ]:
sepal_numeric = pd.to_numeric(iris['sepal_length_cm'], errors='coerce')

print('Typ zmiennej: %s' % sepal_numeric.dtype)

# print(sepal_numeric)

import numpy as np
np.where(sepal_numeric.isna() == True)    # indeksy brakujących wartości 

Sprawdźmy, co było problemem. 

In [ ]:
iris[sepal_numeric.isna()]

Okazuje się, że dwie wartości w pliku zamiast liczby zawierały znak zapytania (`?`).

Wstawmy poprawioną zmienną we właściwe miejsce w danych.

In [ ]:
iris.sepal_length_cm = sepal_numeric
iris.info()

## Uzupełnianie wartości brakujących

Jeżeli zależy nam na zachowaniu pomiarów zawierających braki to możemy spróbować wypełnić je odpowiednimi wartościami, np. takimi, które wydają się najbardziej prawdopodobne, tj. wartością średnią zmiennej lub wartością modalną. Wypełnianie braków zrealizowane może być funkcją ``fillna()``

In [ ]:
sepal_lenght_mean = sepal_numeric.mean()

print('Wartość średnia zmiennej', sepal_lenght_mean)

iris.sepal_length_cm = iris['sepal_length_cm'].fillna(sepal_lenght_mean)

iris.sepal_length_cm.isna().values.any()

## Usuwanie wartości brakujących 

Jednak najczęściej będziemy chcieli się pozbyć pomiarów posiadających brakujące dane. W przypadku, gdy danych jest dostatecznie dużo nie będzie to miało istotnego wpływu na wynik analizy. 

Usuńmy przypadki, które posiadają braki za pomocą metody `dropna()`.


In [ ]:
print("Obecnie w danych jest %d przypadków" % iris.shape[0])

iris = iris.dropna()     # usuwanie wierszy zawierających wartości NaN

print("Po usunięciu braków pozostało %d przypadków" %  iris.shape[0])
iris.isna().any()

## Wartości odstające i inne anomalie


Wypiszmy podstawowe statystyki o danych i sprawdźmy czy występują tam inne anomalie.

In [ ]:
iris.describe()

Dzięki tego typu tabelkom możemy sprawdzić podstawowe informacje o danych oraz sprawdzić czy pomiary  nie przekraczają rozsądnych przedziałów dla poszczególnych cech.

Cecha `sepal_length_cm` posiada podejrzanie małą wartość minimalną (0.055 cm), zaś wartość minimalna zmiennej `sepal_width_cm` jest ujemna (-1.0). Szerokość i wysokość powinny być wartościami większymi od 0. Najprawdopodobniej jest to wartość błędna i ten pomiar należy usunąć.


Wartości odstające najwygodniej wykryć za pomocą wykresu skrzynkowego. 

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sb

sb.boxplot(data=iris);

Skrzynia określa zakres od 1 do 3 kwartyla z kreską środkową oznaczająca medianę (wartość środkowa). Punkty leżące za "wąsami" są podejrzane, są to wartości odstające, nietypowe dla rozkładu. 

Potwierdza się, że dwie pierwsze zmienne posiadają odstające wartości. 

Przypadki odstające tj. takie które znajdują się poza zakresem w które wpada większość danych, nie muszą jednoznacznie być błędne. Niemniej jednak należy im się dobrze przyjrzeć. I bardzo rzadko możemy jednoznacznie stwierdzić, czy taka wartość odstająca była błędem pomiaru, błędem na poziomie akwizycji czy też wprowadzania danych, czy może faktycznie jest odstępstwem od normy w samym fenomenie, który obserwujemy. 

Zobaczmy najpierw, które przypadki posiadają wartość ujemną, gdyż one z pewnością są błędne.

In [ ]:
iris[iris['sepal_width_cm'] < 0.0]

Jest jeden taki przypadek - usuńmy go.

In [ ]:
iris = iris[iris['sepal_width_cm'] > 0.0]

sb.boxplot(data=iris);

Badając rozkład danych przyjrzyjmy się również rozkładowi danych dla każdej pary zmiennych.   
Grupy przypadków zaznaczmy różnymi kolorami.

In [ ]:
import seaborn as sb
import matplotlib.pyplot as plt

sb.pairplot(iris, hue='class')
plt.show()

Z tego wykresu możemy wysnuć od razu następujące wnioski:
- wartości odstające w zmiennej `sepal_length_cm` są bardzo wyraźnie widoczne i dotyczą odmiany Iris-Versicolor,
- domena klasy (`class`) ma 5 unikalnych wartości, podczas gdy powinna mieć ich 3.

Zbadajmy najpierw zmienną `class`. Jest to zmienna kategoryczna.

In [ ]:
# wypiszmy jakie posiaday unikalne wartośći w kolumnie class
print(iris['class'].unique())

iris['class'].value_counts()

Możemy zauważyć, że musiał nastąpić błąd w kodowaniu danych, podczas wprowadzania danych zostały wykonane dwie literówki. Poprawmy to.

In [ ]:
iris.loc[iris['class'] == 'versicolor', 'class'] = 'Iris-versicolor'

# podoby efekt osiągniemy z pomoca metody replace()
iris.loc[:, 'class'] = iris['class'].replace('Iris-setossa', 'Iris-setosa')

print(iris['class'].unique())
print(iris['class'].value_counts())

## Błędy w danych

Przyjrzyjmy się też danym w zmiennej  `sepal_length_cm` odmiany `Iris-versicolor`, które posiadają podejrzanie małe wartości, mniejsze od 2.5cm 

In [ ]:
#dla ułatwienia najpier zapiszę sobie indeks do wykrajania dla tych danych
broken_slice_idx = (iris['class'] == 'Iris-versicolor') & (iris['sepal_length_cm'] < 1.0)

iris.loc[broken_slice_idx]

Możemy wrócić, do tabeli ze statystkami i zauważyć, że wartość `sepal_length_cm` wynosiła średnio 5.6 $\pm$ 1.3. 

In [ ]:
print("%.1f +- %.1f" % (iris.sepal_length_cm.mean() , iris.sepal_length_cm.std()))

Wygląda na to że podczas wprowadzania danych zostały pomylone jednostki miary, dane zostały wprowadzone w milimetrach zamiast w cm. Oczywiście to należałoby w jakiś sposób potwierdzić, ale na ten moment zmieńmy jednostki dla tych danych.

In [ ]:
iris.loc[broken_slice_idx, 'sepal_length_cm'] *= 100.0

In [ ]:
sb.boxplot(data=iris);

In [ ]:
sb.pairplot(iris, hue='class')
plt.show()

## Duplikaty



Dane mogą zawierać powtarzające się pomiary, np. w wyniku nieuwagi pomiary jednego obiektu mogły zostać kilkukrotnie wpisane do bazy danych. W przypadku danych Irys nie mamy pewności, czy powtarzające się dane są wynikami uzyskanymi dla  różnych kwiatów. Spróbujmy jednak poszukać powtarzających się wierszy i je usunąć.

Wypiszmy najpierw wszystkie wiersze, które się duplikują.

In [ ]:
iris_d = iris.duplicated(keep=False)
iris_d

In [ ]:
iris[iris_d]

Argument `keep='first'` metody `duplicated()` pozostawia pierwsze wystąpienie powtarzającego się wiersza i pozwala zindeksować pozostałe duplikaty. Usuńmy wszystkie (oprócz pierwszego) powtarzające się pomiary.

In [ ]:
iris_d = iris.duplicated(keep='first')
print('Ilość powtarzających się przypadków = %d' % (iris_d.sum()))

print('Liczba przypadków przed selekcją = %d' % (iris.shape[0]))
iris = iris.drop_duplicates()
print('Liczba przypadków po odrzuceniu powtarzających się przypadków  = %d' % (iris.shape[0]))

## Standaryzacja danych

Standaryzacja  - normalizacja zmiennych zamieniająca średnią $\mu$ na 0 (centrowanie) a odchylenie standardowe $\sigma$ na 1

$$
z=\frac{x-\mu}{\sigma}
$$

In [ ]:
iris_num = iris.select_dtypes(include=np.number)

iris_std = (iris_num - iris_num.mean()) / iris_num.std()

iris_std.plot(kind='box')
# pd.options.display.float_format = '{:,.2f}'.format
iris_std.describe()

Zwykle przyjmuje się, że wartości odstające leżą dalej niż 3 odchylenia standardowe

In [ ]:
outliers = ((iris_std > 3) | (iris_std < -3)).any(axis=1)

iris[outliers]

## Normalizacja

Normalizacja wartości zmiennych w ustalonym zakresie, zazwyczaj $[-1, 1]$

$$
z= 2 \frac{x - x_{min}}{x_{max}-x_{min}} - 1
$$



In [ ]:
iris_norm = 2 * (iris_num - iris_num.min()) / (iris_num.max() - iris_num.min()) - 1

iris_norm.plot(kind='box')
iris_norm.describe()

## Próbkowanie (sampling)

In [ ]:
sample = iris.sample(n=10) # wybieramy losowo 10 próbek
sample

In [ ]:
sample = iris.sample(frac=0.05, random_state=13) # wybieramy losowo 5% próbek z całego zbioru
sample

In [ ]:
sample = iris.sample(frac=0.05, replace=True, random_state=13) # wybieramy losowo 5% próbek, ale ta sama próbka może być wybrana wiele razy
sample

## Dyskretyzacja danych

Zamiana zmiennych ciągłych na dyskretne

In [ ]:
iris['sepal_length_cm'].hist(bins=10);               # podział zbioru na 10 elementów

In [ ]:
bins = pd.cut(iris['sepal_length_cm'], 3)   # podział zbioru na 3 elementy o równych odstępach (mniej-więcej)

bins.value_counts(sort=False)

In [ ]:
bins = pd.qcut(iris['sepal_length_cm'], 4, labels=['a', 'b', 'c', 'd']) # podział zbioru na 4 podzbiory o zbliżonej liczebności 
bins.value_counts(sort=False)

In [ ]:
iris['sepal_length_size'] = pd.cut(iris['sepal_length_cm'], 3, labels=['small', 'medium', 'large'])
iris.info()

In [ ]:
iris['sepal_length_cm'].groupby(iris['sepal_length_size']).mean()

## Zamiana kategorycznych danych na zmienne numeryczne
### Kodowanie one-hot

Kodowanie *one-hot* - zamiana wartości kategorycznych na wektor binarny $[0, 0, 1, 0, \ldots, 0]$


In [ ]:
class_one_hot = pd.get_dummies(iris['class'])
class_one_hot

### Mapowanie wartości kategorycznych na liczby

In [ ]:
size_map = {
    'small' : 1,
    'medium' : 2,
    'large' : 3
}

size_data = iris['sepal_length_size'].map(size_map)
size_data

## Zadanie

Wczytaj dane "Breast Cancer Wisconsin" i przeprowadź preprocesing zgodnie z podanymi poniżej wytycznymi.

Dane znajdują się w repozytorium pod adresem ``dane/breast-cancer.data``. Można je tez pobrać z adresu https://www.is.umk.pl/~grochu/wdm/files/breast-cancer.data

Dane zawierają wartości opisujące cechy jąder komórkowych obecnych na obrazie uzyskanym przy badaniu piersi dla dwóch grup badanych: `benign` (złośliwy), `malignat` (łagodny).

Oto lista zmiennych:

```
   #  Attribute                     Domain
   -- -----------------------------------------
   1. Sample code number            id number
   2. Clump Thickness               1 - 10
   3. Uniformity of Cell Size       1 - 10
   4. Uniformity of Cell Shape      1 - 10
   5. Marginal Adhesion             1 - 10
   6. Single Epithelial Cell Size   1 - 10
   7. Bare Nuclei                   1 - 10
   8. Bland Chromatin               1 - 10
   9. Normal Nucleoli               1 - 10
  10. Mitoses                       1 - 10
  11. Class                         (2 for benign, 4 for malignant)

```

Wszystkie istotne cechy posiadają wartości numeryczne z zakresu od 1 do 10, ostatnia zmienna zawiera informacje o 2 klasach.

1. Wczytaj zbiór danych ``brast-cancer.data`` używając Pandas. Dane są w formacie zgodnym z CSV (wartości oddzielone przecinkami). Zwróć uwagę na to, że plik nie posiada nagłówka, tzn. pierwsza linia pliku nie zawiera nazw zmiennych. Uzupełnij nazwy zmiennych (kolumn) zgodne z listą podanych wyżej atrybutów.

2. Pierwsza zmienna zawiera liczbę porządkową (``Sample_code_number``), unikatową dla każdego badanego. Jest ona nieistotna dla analizy. Usuń ją ze zbioru.

3. Wartości brakujące w pliku wejściowym kodowane są za pomocą znaku zapytania (`?`). Sprawdź dla ilu badanych występują wartości brakujące i w których zmiennych występują. 

4. Zastąp wartości brakujące wartością modalną (zob. funkcja [mode()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.mode.html)). W przypadku występowania wielu wartości modalnych użyj dowolnej z nich. Po transformacji wszystkie zmienne powinny być zmiennymi numerycznymi.

5. Sprawdź, czy zbiór danych zawiera przypadki odstające lub nietypowe. Spodziewamy się, że wszystkie zmienne (oprócz klasy) mają wartości w zakresie od 1 do 10. Jeżeli uznasz, że pewne wartości zmiennych są niespójne ze zbiorem danych to usuń przypadki zawierające te wartości ze zbioru danych. 

6. Usuń ze zbioru przypadki odstające, których wartości zmiennych leżą poza przedziałem $(\bar{x}-3\cdot\sigma, \bar{x}+ 3\cdot\sigma)$, gdzie $\bar{x}$ to wartość średnia cechy, $\sigma$ to odchylnie standardowe.

7. Sprawdź czy dane zawierają powtarzające się pomiary i usuń ze zbioru danych duplikaty.